# Data Pre-processing
In this first notebook I'm running the data pre-processinh phase, where I'm loading, merging and formatting the different sources of information for our regression problem.

**Author**: Arthur G.
***

## Importing Dependencies
In this first section I'm importing all the dependencies for the data pre-processing phase.

In [1]:
# adding custom functions (project specific, from src)
import sys
sys.path.append('../')

# dependencies
import os
import numpy as np
import pandas as pd
from unidecode import unidecode
from src.features.build_features import *

# settings
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.precision', 3)

## Loading Datasets
In this section I'm loading the different sources of data for our modeling phase.

In [2]:
data2015 = pd.read_excel(os.path.join('..', 'data', 'raw', 'ships2015.xlsx'))
data2017 = pd.read_csv(os.path.join('..', 'data', 'raw', 'ships2017.csv'))

print(f"2015 data shape: {data2015.shape} | 2017 data shape: {data2017.shape}")

2015 data shape: (691, 130) | 2017 data shape: (652, 133)


### Formatting 2015 Data
Here I'm applying some data pre-processing steps on the data collected in 2015. The same pre-processing will be applied on the 2017 dataset.

In [3]:
# filtering important columns only
data_2015_col_filter = [
    'ITEM_22_01', 'ITEM_22_02',
    '2.2. Escalas', 'Outro. Tipo de embarcaçao', '8. Material do casco',
    '9. Comprimento', '10. Boca', '11. Calado', '12. Pontal', '13. Potência do motor',
    '14. Velocidade', '15. Número de propulsores','18. Número de geradores', 'ITEM_19_01', 
    'ITEM_19_02', 'ITEM_19_03', 'ITEM_24_01', 'ITEM_24_02'
]
data2015 = data2015[data_2015_col_filter]

print(f"2015 data shape: {data2015.shape}")

2015 data shape: (691, 18)


Now I'm pre-processing, renaming and organizing columns from the 2015 dataset.

In [4]:
# creating the target column
data2015['FUEL_CONSUMPTION'] = data2015.ITEM_24_01 + data2015.ITEM_24_02

# removing useless columns
data2015.drop(['ITEM_24_01', 'ITEM_24_02'], axis=1, inplace=True)

# renaming columns
data2015.rename(
    columns={
        '2.2. Escalas': 'STOPOVERS', 'Outro. Tipo de embarcaçao': 'VESSEL_TYPE', 
        '8. Material do casco': 'HULL_MATERIAL','9. Comprimento': 'VESSEL_LENGTH', 
        '10. Boca': 'VESSEL_BEAN', '11. Calado': 'VESSEL_DRAFT', '12. Pontal': 'VESSEL_DEPTH', 
        '13. Potência do motor': 'MOTOR_POWER', '14. Velocidade': 'SPEED', 
        '15. Número de propulsores': 'NUM_PROPELLERS', '18. Número de geradores': 'NUM_GENERATORS', 
        'ITEM_19_01': 'PASS_CAPACITY', 'ITEM_19_02': 'LOAD_CAPACITY', 'ITEM_19_03': 'CREW', 
        'ITEM_22_01': 'DEPARTURE_ARRIVAL_DURATION', 'ITEM_22_02': 'ARRIVAL_DEPARTURE_DURATION'
    },
    inplace = True
)

data2015.head(3)

,DEPARTURE_ARRIVAL_DURATION,ARRIVAL_DEPARTURE_DURATION,STOPOVERS,VESSEL_TYPE,HULL_MATERIAL,VESSEL_LENGTH,VESSEL_BEAN,VESSEL_DRAFT,VESSEL_DEPTH,MOTOR_POWER,SPEED,NUM_PROPELLERS,NUM_GENERATORS,PASS_CAPACITY,LOAD_CAPACITY,CREW,FUEL_CONSUMPTION
0,02:15:00,02:15:00,-,Lancha,Aço Naval,30.0,6.5,2.2,3.1,500,25.0,1.0,1.0,220.0,0.0,5.0,180.0
1,10:00:00,10:00:00,SÃO SEBASTIÃO DA BOA VISTA,Passageiro/Carga geral,Madeira,34.0,8.4,2.2,3.0,600,16.0,2.0,2.0,280.0,143.0,5.0,300.0
2,12:00:00,12:00:00,-,Passageiro/Carga geral,Madeira,34.0,8.0,2.1,2.9,550,22.0,1.0,1.0,180.0,144.0,6.0,1600.0


Now I'll be preprocessing columns in need with our custom functions.

In [5]:
# removing nans
data2015.dropna(inplace=True)

# converting DEPARTURE_ARRIVAL_DURATION and ARRIVAL_DEPARTURE_DURATION in seconds
data2015.DEPARTURE_ARRIVAL_DURATION = data2015.DEPARTURE_ARRIVAL_DURATION.astype(str)
data2015.ARRIVAL_DEPARTURE_DURATION = data2015.ARRIVAL_DEPARTURE_DURATION.astype(str)
data2015['DEPARTURE_ARRIVAL_DURATION'] = data2015.DEPARTURE_ARRIVAL_DURATION.map(lambda x: handle_travel_time(x))
data2015['ARRIVAL_DEPARTURE_DURATION'] = data2015.ARRIVAL_DEPARTURE_DURATION.map(lambda x: handle_travel_time(x))
data2015['DEPARTURE_ARRIVAL_DURATION'] = data2015.DEPARTURE_ARRIVAL_DURATION.map(lambda x: time_str_2_sec(x))
data2015['ARRIVAL_DEPARTURE_DURATION'] = data2015.ARRIVAL_DEPARTURE_DURATION.map(lambda x: time_str_2_sec(x))

# converting CREW, NUM_GENERATORS and NUM_PROPELLERS from float to int
data2015.CREW = data2015.CREW.astype(int)
data2015.NUM_GENERATORS = data2015.NUM_GENERATORS.astype(int)
data2015.NUM_PROPELLERS = data2015.NUM_PROPELLERS.astype(int)

# converting the STOPOVERS column from string to the count of STOPOVERS
data2015.STOPOVERS = data2015.STOPOVERS.map(lambda x: count_stopovers(x)).astype(int)

Now I'll translate the columns VESSEL_TYPE and HULL_MATERIAL from brazilian portuguese to english.

In [6]:
# setting translations
translate_vessel_type = {
    'Passageiro/Carga geral': 'PASSENGER/GENERAL_LOAD',
    'Lancha': 'MOTORBOAT',
    'Ferry Boat': 'FERRY_BOAT',
    'Passageiro': 'PASSENGERS',
    'Chata': 'FLAT_BOTTOMED_BOAT',
    'Catamarã Passageiro': 'CATAMARAN',
    'Outro': 'OTHER'
}

translate_hull_material = {
    'Madeira': 'WOOD',
    'Aço Naval': 'NAVAL_STEEL',
    'Alumínio': 'ALUMINIUM',
    'Fibra': 'FIBER'
}

# applying translations
data2015.VESSEL_TYPE = data2015.VESSEL_TYPE.map(lambda x: translate_vessel_type[x])
data2015.HULL_MATERIAL = data2015.HULL_MATERIAL.map(lambda x: translate_hull_material[x])

data2015.head()

,DEPARTURE_ARRIVAL_DURATION,ARRIVAL_DEPARTURE_DURATION,STOPOVERS,VESSEL_TYPE,HULL_MATERIAL,VESSEL_LENGTH,VESSEL_BEAN,VESSEL_DRAFT,VESSEL_DEPTH,MOTOR_POWER,SPEED,NUM_PROPELLERS,NUM_GENERATORS,PASS_CAPACITY,LOAD_CAPACITY,CREW,FUEL_CONSUMPTION
0,8100.0,8100.0,0,MOTORBOAT,NAVAL_STEEL,30.0,6.5,2.20,3.1,500,25.0,1,1,220.0,0.0,5,180.0
1,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.4,2.20,3.0,600,16.0,2,2,280.0,143.0,5,300.0
2,43200.0,43200.0,0,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.0,2.10,2.9,550,22.0,1,1,180.0,144.0,6,1600.0
3,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,NAVAL_STEEL,42.0,10.9,1.36,1.6,550,22.0,2,2,436.0,300.0,6,2550.0
4,46800.0,39600.0,2,MOTORBOAT,ALUMINIUM,27.0,3.6,0.70,1.5,660,20.0,1,1,70.0,46.0,5,3600.0


### Formatting 2017 Data
Here I'm applying the same pre-processing steps created for the data above.

In [7]:
# filtering important columns only
data_2017_col_filter = [
    '22.1 Tempo de duração da viagem entre (Origem - Destino)', 
    '22.2 Tempo de duração da viagem entre (Destino - Origem)', 
    '2.2. Escalas', '7. Tipo de embarcaçao', '8. Material do casco',
    '9. Comprimento', '10. Boca', '11. Calado', '12. Pontal', '13. Potência do motor',
    '14. Velocidade', '15. Número de propulsores', '16. TPB', '17. Deslocamento leve', 
    '18. Número de geradores', '19.1 Número de Passageiros', '19.2 Capacidade de Carga (t)', 
    '19.3 Número de Tripulantes', '24.1 Consumo de Combustível (L) (Origem - Destino)', 
    '24.2 Consumo de Combustível (L) (Destino - Origem)'
]

data2017 = data2017[data_2017_col_filter]

# creating the target column
data2017['FUEL_CONSUMPTION'] = data2017['24.1 Consumo de Combustível (L) (Origem - Destino)'] + data2017['24.2 Consumo de Combustível (L) (Destino - Origem)']

# removing useless columns
data2017.drop(
    [
        '24.1 Consumo de Combustível (L) (Origem - Destino)', 
        '24.2 Consumo de Combustível (L) (Destino - Origem)'
    ], 
    axis=1, 
    inplace=True
)

# renaming columns
data2017.rename(
    columns={
        '2.2. Escalas': 'STOPOVERS', '7. Tipo de embarcaçao': 'VESSEL_TYPE', 
        '8. Material do casco': 'HULL_MATERIAL','9. Comprimento': 'VESSEL_LENGTH', 
        '10. Boca': 'VESSEL_BEAN', '11. Calado': 'VESSEL_DRAFT', '12. Pontal': 'VESSEL_DEPTH', 
        '13. Potência do motor': 'MOTOR_POWER', '14. Velocidade': 'SPEED', '16. TPB': 'DWT',
        '17. Deslocamento leve': 'LIGHT_DISPLACEMENT', '15. Número de propulsores': 'NUM_PROPELLERS', 
        '18. Número de geradores': 'NUM_GENERATORS', '19.1 Número de Passageiros': 'PASS_CAPACITY', 
        '19.2 Capacidade de Carga (t)': 'LOAD_CAPACITY', '19.3 Número de Tripulantes': 'CREW', 
        '22.1 Tempo de duração da viagem entre (Origem - Destino)': 'DEPARTURE_ARRIVAL_DURATION', 
        '22.2 Tempo de duração da viagem entre (Destino - Origem)': 'ARRIVAL_DEPARTURE_DURATION'
    },
    inplace = True
)

# converting DEPARTURE_ARRIVAL_DURATION and ARRIVAL_DEPARTURE_DURATION from hours to seconds
data2017.DEPARTURE_ARRIVAL_DURATION = data2017.DEPARTURE_ARRIVAL_DURATION * 60
data2017.ARRIVAL_DEPARTURE_DURATION = data2017.ARRIVAL_DEPARTURE_DURATION * 60

# applying translations on VESSEL_TYPE and HULL_MATERIAL
translate_vessel_type = {
    'PASSAGEIRO/CARGA GERAL': 'PASSENGER/GENERAL_LOAD',
    'LANCHA': 'MOTORBOAT',
    'FERRY BOAT': 'FERRY_BOAT',
    'PASSAGEIRO': 'PASSENGERS',
    'CHATA': 'FLAT_BOTTOMED_BOAT',
    'CATAMARA PASSAGEIRO': 'CATAMARAN',
    'OUTRO': 'OTHER'
}

translate_hull_material = {
    'MADEIRA': 'WOOD',
    'AÇO NAVAL': 'NAVAL_STEEL',
    'ALUMÍNIO': 'ALUMINIUM',
    'FIBRA': 'FIBER'
}

data2017.VESSEL_TYPE = data2017.VESSEL_TYPE.map(lambda x: translate_vessel_type[x])
data2017.HULL_MATERIAL = data2017.HULL_MATERIAL.map(lambda x: translate_hull_material[x])

# converting the STOPOVERS column from string to the count of STOPOVERS
data2017.STOPOVERS = data2017.STOPOVERS.map(lambda x: count_stopovers(x, version='2017')).astype(int)

# dropping nans
data2017.dropna(inplace=True)

data2017.head()

,DEPARTURE_ARRIVAL_DURATION,ARRIVAL_DEPARTURE_DURATION,STOPOVERS,VESSEL_TYPE,HULL_MATERIAL,VESSEL_LENGTH,VESSEL_BEAN,VESSEL_DRAFT,VESSEL_DEPTH,MOTOR_POWER,SPEED,NUM_PROPELLERS,DWT,LIGHT_DISPLACEMENT,NUM_GENERATORS,PASS_CAPACITY,LOAD_CAPACITY,CREW,FUEL_CONSUMPTION
0,180.0,180.0,0,PASSENGER/GENERAL_LOAD,WOOD,15.00,3.70,1.00,1.2,135,15,1,15,3,1.0,22,10,2,80
1,1200.0,1200.0,1,PASSENGER/GENERAL_LOAD,WOOD,31.45,7.60,1.80,2.2,550,17,2,185,40,3.0,192,150,7,2000
2,1440.0,1440.0,0,PASSENGER/GENERAL_LOAD,WOOD,26.20,6.55,1.70,2.1,420,10,1,110,20,1.0,70,90,4,1800
3,300.0,240.0,0,PASSENGER/GENERAL_LOAD,WOOD,20.50,5.40,1.85,2.1,200,18,1,40,8,2.0,68,36,3,250
4,480.0,480.0,2,MOTORBOAT,ALUMINIUM,18.30,2.80,1.80,2.0,200,18,1,20,5,0.0,90,0,3,1000


## Merging Data Sources
In this last section I'm merging the two pre-processed data sources.

Let's make the datasets ready to merge to each other. We need to add two more columns on the dataset from 2015, DWT and LIGHT_DISPLACEMENT, once this dataset had not any observation for these two columns.

In [8]:
data2015['DWT'] = np.nan
data2015['LIGHT_DISPLACEMENT'] = np.nan

data2015 = data2015[[
    'DEPARTURE_ARRIVAL_DURATION', 'ARRIVAL_DEPARTURE_DURATION', 'STOPOVERS',
    'VESSEL_TYPE', 'HULL_MATERIAL', 'VESSEL_LENGTH', 'VESSEL_BEAN',
    'VESSEL_DRAFT', 'VESSEL_DEPTH', 'MOTOR_POWER', 'SPEED',
    'NUM_PROPELLERS', 'NUM_GENERATORS', 'DWT', 'LIGHT_DISPLACEMENT',
    'CREW', 'PASS_CAPACITY', 'LOAD_CAPACITY', 'FUEL_CONSUMPTION'
]]

# handling missing data on the MOTOR_POWER column
data2015.drop(index=data2015.query("MOTOR_POWER == 'Não sabe/não informado'").index, inplace=True)
data2015.reset_index(drop=True, inplace=True)
data2015.MOTOR_POWER = data2015.MOTOR_POWER.astype(int)

data2015.head()

,DEPARTURE_ARRIVAL_DURATION,ARRIVAL_DEPARTURE_DURATION,STOPOVERS,VESSEL_TYPE,HULL_MATERIAL,VESSEL_LENGTH,VESSEL_BEAN,VESSEL_DRAFT,VESSEL_DEPTH,MOTOR_POWER,SPEED,NUM_PROPELLERS,NUM_GENERATORS,DWT,LIGHT_DISPLACEMENT,CREW,PASS_CAPACITY,LOAD_CAPACITY,FUEL_CONSUMPTION
0,8100.0,8100.0,0,MOTORBOAT,NAVAL_STEEL,30.0,6.5,2.20,3.1,500,25.0,1,1,NaN,NaN,5,220.0,0.0,180.0
1,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.4,2.20,3.0,600,16.0,2,2,NaN,NaN,5,280.0,143.0,300.0
2,43200.0,43200.0,0,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.0,2.10,2.9,550,22.0,1,1,NaN,NaN,6,180.0,144.0,1600.0
3,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,NAVAL_STEEL,42.0,10.9,1.36,1.6,550,22.0,2,2,NaN,NaN,6,436.0,300.0,2550.0
4,46800.0,39600.0,2,MOTORBOAT,ALUMINIUM,27.0,3.6,0.70,1.5,660,20.0,1,1,NaN,NaN,5,70.0,46.0,3600.0


For the dataset from 2017 we just need to reorganize the columns and handle some missing values.

In [9]:
data2017 = data2017[[
    'DEPARTURE_ARRIVAL_DURATION', 'ARRIVAL_DEPARTURE_DURATION', 'STOPOVERS',
    'VESSEL_TYPE', 'HULL_MATERIAL', 'VESSEL_LENGTH', 'VESSEL_BEAN',
    'VESSEL_DRAFT', 'VESSEL_DEPTH', 'MOTOR_POWER', 'SPEED',
    'NUM_PROPELLERS', 'NUM_GENERATORS', 'DWT', 'LIGHT_DISPLACEMENT',
    'CREW', 'PASS_CAPACITY', 'LOAD_CAPACITY', 'FUEL_CONSUMPTION'
]]

# handling missing nans
for idx in list(data2017.query("DWT == 'Não informado/Não sabe'").index):
    data2017.loc[idx, 'DWT'] = np.nan
    
for idx in list(data2017.query("LIGHT_DISPLACEMENT == 'Não informado/Não sabe'").index):
    data2017.loc[idx, 'LIGHT_DISPLACEMENT'] = np.nan
    
for idx in list(data2017.query("MOTOR_POWER == 'Não sabe/não informado'").index):
    data2017.loc[idx, 'MOTOR_POWER'] = np.nan
    
# handling data types
data2017.MOTOR_POWER = data2017.MOTOR_POWER.astype(float)
data2017.DWT = data2017.DWT.astype(float)
data2017.LIGHT_DISPLACEMENT = data2017.LIGHT_DISPLACEMENT.astype(float)

Merging the datasets.

In [10]:
# merging the datasets
data = pd.concat([data2015, data2017], ignore_index=True)

# handling generated nans
data.dropna(subset=['MOTOR_POWER'], axis=0, inplace=True)

# dropping rows without travel time
data.drop(index=data.query("DEPARTURE_ARRIVAL_DURATION == 0 & ARRIVAL_DEPARTURE_DURATION == 0").index, inplace=True)
data.reset_index(drop=True, inplace=True)

# filling rows without DEPARTURE_ARRIVAL_DURATION with its ARRIVAL_DEPARTURE_DURATION and vice versa
for idx in list(data.query("DEPARTURE_ARRIVAL_DURATION == 0").index):
    data.iloc[idx, 0] = data.iloc[idx, 1]
    
for idx in list(data.query("ARRIVAL_DEPARTURE_DURATION == 0").index):
    data.iloc[idx, 1] = data.iloc[idx, 0]
    
# resetting index
data.reset_index(drop=True, inplace=True)
    
data.head()

,DEPARTURE_ARRIVAL_DURATION,ARRIVAL_DEPARTURE_DURATION,STOPOVERS,VESSEL_TYPE,HULL_MATERIAL,VESSEL_LENGTH,VESSEL_BEAN,VESSEL_DRAFT,VESSEL_DEPTH,MOTOR_POWER,SPEED,NUM_PROPELLERS,NUM_GENERATORS,DWT,LIGHT_DISPLACEMENT,CREW,PASS_CAPACITY,LOAD_CAPACITY,FUEL_CONSUMPTION
0,8100.0,8100.0,0,MOTORBOAT,NAVAL_STEEL,30.0,6.5,2.20,3.1,500.0,25.0,1,1.0,NaN,NaN,5,220.0,0.0,180.0
1,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.4,2.20,3.0,600.0,16.0,2,2.0,NaN,NaN,5,280.0,143.0,300.0
2,43200.0,43200.0,0,PASSENGER/GENERAL_LOAD,WOOD,34.0,8.0,2.10,2.9,550.0,22.0,1,1.0,NaN,NaN,6,180.0,144.0,1600.0
3,36000.0,36000.0,1,PASSENGER/GENERAL_LOAD,NAVAL_STEEL,42.0,10.9,1.36,1.6,550.0,22.0,2,2.0,NaN,NaN,6,436.0,300.0,2550.0
4,46800.0,39600.0,2,MOTORBOAT,ALUMINIUM,27.0,3.6,0.70,1.5,660.0,20.0,1,1.0,NaN,NaN,5,70.0,46.0,3600.0


Serializing the final dataset.

In [11]:
data.to_csv("../data/interim/fuel_consumption_data.csv", index=False)